In [60]:
# Reference used: https://www.geeksforgeeks.org/tf-idf-for-bigrams-trigrams/
import nltk 
import re 
import math
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import normalize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.util import ngrams
import pandas as pd   
import jsonlines
import json
from tqdm.notebook import tqdm
import pickle

In [61]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\RAGHAV\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\RAGHAV\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [62]:
documents = []
docID = {}
term_total_citations = {}
author_term_citations = {}
idx = 0
with jsonlines.open('data_india_sample.jl') as reader:
    for obj in tqdm(reader, leave=False):
        documents.append(obj)
        docID[idx] = obj['user']
        author_term_citations[obj['user']] = {}
        idx += 1

0it [00:00, ?it/s]

In [63]:
# Code Taken From: https://www.geeksforgeeks.org/tf-idf-for-bigrams-trigrams/
def remove_string_special_characters(s):
    # removes special characters with ' '
    stripped = re.sub('[^a-zA-z\s]', ' ', s)
#     print(stripped)
    stripped = re.sub('_', ' ', stripped)
      
    # Change any white space to one space
    stripped = re.sub('\s+', ' ', stripped)
      
    # Remove start and end white spaces
    stripped = stripped.strip()
    if stripped != '':
            return stripped.lower()

In [64]:
# Code Reference: https://www.geeksforgeeks.org/python-stemming-words-with-nltk/
def stem_string(sentence):
    ps = PorterStemmer()
    
    words = word_tokenize(sentence)
    words = [ps.stem(word) for word in words]
    return " ".join(words)

In [65]:
def remove_stop_words(sentence):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(sentence)
    words = [word for word in words if word not in stop_words]
    return " ".join(words)

In [66]:
def process_string(sentence):
    sentence = remove_string_special_characters(sentence)
    sentence = stem_string(sentence)
    sentence = remove_stop_words(sentence)
    return sentence

### TF-IDF Matrix

In [67]:
def update_term_citations(document, sentence, numCitations, userID):
    numCitations += 1
    sentence = sentence.lower()
    sentence = re.sub(r'[^a-zA-Z0-9\s]', ' ', sentence)
    tokens = [token for token in sentence.split(" ") if token != ""]
    for n in range(1, 5):
#         seen_ngms = []
        ngms = list(ngrams(tokens, n))
        for ngm in ngms:
            gm = " ".join(ngm)
#             if gm in seen_ngms:
#                 continue
#             seen_ngms.append(gm)
            if gm in author_term_citations[userID]:
                author_term_citations[userID][gm] += numCitations
            else:
                author_term_citations[userID][gm] = numCitations
            if gm in term_total_citations:
                term_total_citations[gm] += numCitations
            else:
                term_total_citations[gm] = numCitations

In [68]:
word_documents = []
for document in tqdm(documents, leave=False):
    sentences = []
    for subject in document['subjects']:
        sentence = process_string(subject)
        sentences.append(sentence)
        update_term_citations(document, sentence, int(document['citationsAll']), document['user'])
    for paper in document['papers']:
        try:
            sentence = paper['title'] + " " + paper['conference']
            sentence = process_string(sentence)
        except: 
            continue
#         print(sentence)
#         print(sentence)
        update_term_citations(document, sentence, paper['citations'], document['user'])
        sentences.append(sentence)

#     word_documents.append(sentences)
    word_documents.append(".\n".join(sentences))
#     print(word_documents[-1])
#     word_documents.append(sentences)
#     print(".".join(sentences))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [69]:
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,4),min_df=0, analyzer='word')
tfidf_matrix = vectorizer.fit_transform(word_documents) 
features = (vectorizer.get_feature_names())
idf_scores = vectorizer.idf_

cnt_vct = CountVectorizer(stop_words='english', ngram_range=(1,4),min_df=0, analyzer='word')
cnt_matrix = cnt_vct.fit_transform(word_documents)
cnt_features = cnt_vct.get_feature_names()

cnt_matrix_arr = cnt_matrix.toarray()
tfidf_new_matrix = cnt_matrix_arr*idf_scores

# print(features)

In [54]:
# print(author_term_citations)
# feature_idx = {}
# for idx, feature in enumerate(features):
#     feature_idx[feature] = idx
# author_idx = 0
# for author, author_dict in tqdm(author_term_citations.items(), leave=False):
#     for term, term_citations in author_dict.items():
#         if term in feature_idx:
#             term_idx = feature_idx[term]
#             log_factor = math.log( 1 + ((term_citations) / (term_total_citations[term] + 1)) , 2)
# #             print(log_factor)
#             tfidf_new_matrix[author_idx, term_idx] *= log_factor
#     author_idx += 1

  0%|          | 0/994 [00:00<?, ?it/s]

In [55]:
# tfidf_matrix = normalize(tfidf_matrix, norm='l2', axis=0)

# Activeness 

In [24]:
maxrng=20
minyr,maxyr=2001,2020
mat_activeness = np.zeros((len(documents),maxrng))
ind=0
for document in tqdm(documents, leave=False):
    papers=document['papers']
    for paper in papers:
        paperyr=paper['year']
        if paperyr>=minyr and paperyr<=maxyr:
            dis=maxyr-paperyr
            mat_activeness[ind,dis]+=paper['citations']
    ind+=1

ind=0
for i in range(mat_activeness.shape[0]):
    for j in range(mat_activeness.shape[1]):
        if j>0:
            mat_activeness[i,j]+=mat_activeness[i,j-1]
            
# print(mat_activeness)
            

  0%|          | 0/10 [00:00<?, ?it/s]

[[  0.   0.  66.  89. 111. 186. 303. 313. 376. 385. 408. 485. 485. 485.
  485. 485. 485. 485. 485. 485.]
 [ 32.  32.  37.  37.  37.  37.  45.  45.  61.  61.  61.  68.  68.  68.
   68.  68.  68.  68.  68.  68.]
 [ 24.  24.  25.  25.  25.  25.  25.  25.  25.  25.  25.  25.  25.  25.
   25.  25.  25.  25.  25.  25.]
 [  0.   0.   0.   0.   5.   6.  18.  51.  51.  51.  51.  51.  51.  51.
   51.  51.  51.  51.  51.  51.]
 [  3.   3.   3.   3.   3.   3.   3.   3.   3.   3.   3.   3.   3.   3.
    3.   3.   3.   3.   3.   3.]
 [  0.   0.   0.   0.   0.   0.   0.   3.   3.   3.   3.   3.   3.   3.
    3.   3.   3.   3.   3.   3.]
 [  5.   5.   5.   5.   5.   5.   8.   8.   8.   8.   8.   8.   8.   8.
    8.   8.   8.   8.   8.   8.]
 [ 25. 117. 239. 273. 417. 442. 442. 442. 442. 442. 442. 442. 442. 442.
  442. 442. 442. 442. 442. 442.]
 [  0.   0.  13.  13.  23.  26.  26.  26.  51.  51.  51.  51.  51.  51.
   51.  51.  51.  51.  51.  51.]
 [  3.  23.  23.  32.  57. 177. 194. 243. 245. 245. 245

## Slope of citations

In [25]:
# Columns -> slope of citations, hindex, institute reputation
other_mets = np.zeros((len(documents),3))

# H index
ind=0
for document in tqdm(documents, leave=False):
    other_mets[ind,1]=document["h-indexAll]
    ind+=1
                               
# Slope of citations (omitting current year)
# ref: https://www.varsitytutors.com/hotmath/hotmath_help/topics/line-of-best-fit
ind=0
for document in tqdm(documents, leave=False):
    for entry in document['yearCitations']:
        vals=[]
        vals.append(entry['citations'])
        if ind==0:
            print(vals)
#     other_mets[ind,0]=slope_calc(document["h-indexAll])
    ind+=1                     


                               

SyntaxError: EOL while scanning string literal (<ipython-input-25-ff368d904fcb>, line 7)

### Pickle the Matrix and Vectorizer

In [13]:
# !mkdir -p web_data

In [70]:
pickle.dump(cnt_vct, open("web_data/count_vectorizer.pkl", "wb"))
pickle.dump(tfidf_new_matrix, open("web_data/tfidf_new_matrix.pkl", "wb"))
pickle.dump(docID, open("web_data/doc_id_dict.pkl", "wb"))

### Query Subjects

In [71]:
query="machine learning"
# print(features[3730])
query = process_string(query)
cnt_matrix_test = cnt_vct.transform([query])
cnt_matrix_test_arr = cnt_matrix_test.toarray()

res = np.matmul(tfidf_new_matrix,cnt_matrix_test_arr.T).squeeze()
related_docs_indices = res.argsort()[:-10:-1]
related_results = [docID[i] for i in related_docs_indices]
print(related_results)

['vVRbl4QAAAAJ', 'a2Wgu0EAAAAJ', '0MNT3h8AAAAJ', 'qBUNlhUAAAAJ', 'pIkLx0AAAAAJ', 'inrwXiIAAAAJ', 'kB1K2McAAAAJ', 'kB1K2McAAAAJ', 'OVjv4SUAAAAJ']


### Pickled Testing

In [72]:
cnt_vct = pickle.load(open("web_data/count_vectorizer.pkl", "rb"))
tfidf_new_matrix = pickle.load(open("web_data/tfidf_new_matrix.pkl", "rb"))
docID = pickle.load(open("web_data/doc_id_dict.pkl", "rb"))

In [73]:
query="machine learning"
# print(features[3730])
query = process_string(query)
cnt_matrix_test = cnt_vct.transform([query])
cnt_matrix_test_arr = cnt_matrix_test.toarray()

res = np.matmul(tfidf_new_matrix,cnt_matrix_test_arr.T).squeeze()
related_docs_indices = res.argsort()[:-10:-1]
related_results = [docID[i] for i in related_docs_indices]
print(related_results)

['vVRbl4QAAAAJ', 'a2Wgu0EAAAAJ', '0MNT3h8AAAAJ', 'qBUNlhUAAAAJ', 'pIkLx0AAAAAJ', 'inrwXiIAAAAJ', 'kB1K2McAAAAJ', 'kB1K2McAAAAJ', 'OVjv4SUAAAAJ', 'SkUNteYAAAAJ']
